# The Canonical Time-series Characteristics (catch22) transform

catch22\[1\] is a collection of 22 time series features extracted from the 7000+ present in the _hctsa_\[2\]\[3\] toolbox.
A hierarchical clustering was performed on the correlation matrix of features that performed better than random chance to remove redundancy.
These clusters were sorted by balanced accuracy using a decision tree classifier and a single feature was selected from the 22 clusters formed, taking into account balanced accuracy results, computational efficiency and interpretability.

In this notebook, we will demonstrate how to use the catch22 transformer on the ItalyPowerDemand univariate and BasicMotions multivariate datasets. We also show catch22 used for classication with a random forest classifier.

Both make use of the features implemented in the catch22 package (<https://github.com/chlubba/catch22>), where versions of catch22 for C and MATLAB are also available.

<span style="color:red">**catch22 does not support Windows OS currently.**</span>

#### References:

\[1\] Lubba, C. H., Sethi, S. S., Knaute, P., Schultz, S. R., Fulcher, B. D., & Jones, N. S. (2019). catch22: CAnonical Time-series CHaracteristics. Data Mining and Knowledge Discovery, 33(6), 1821-1852.

\[2\] Fulcher, B. D., & Jones, N. S. (2017). hctsa: A computational framework for automated time-series phenotyping using massive feature extraction. Cell systems, 5(5), 527-531.

\[3\] Fulcher, B. D., Little, M. A., & Jones, N. S. (2013). Highly comparative time-series analysis: the empirical structure of time series and their methods. Journal of the Royal Society Interface, 10(83), 20130048.

## 1. Imports

In [1]:
from sktime.transformers.panel.catch22_features import Catch22
from sktime.classification.hybrid._catch22_forest_classifier \
    import Catch22ForestClassifier

from sklearn import metrics
from sktime.datasets import load_italy_power_demand, load_basic_motions

## 2. Load data

In [2]:
IPD_X_train, IPD_y_train = load_italy_power_demand(split='train', return_X_y=True)
IPD_X_test, IPD_y_test = load_italy_power_demand(split='test', return_X_y=True)

print(IPD_X_train.shape, IPD_y_train.shape, IPD_X_test.shape, IPD_y_test.shape)

BM_X_train, BM_y_train = load_basic_motions(split='train', return_X_y=True)
BM_X_test, BM_y_test = load_basic_motions(split='test', return_X_y=True)

print(BM_X_train.shape, BM_y_train.shape, BM_X_test.shape, BM_y_test.shape)

(67, 1) (67,) (1029, 1) (1029,)
(40, 6) (40,) (40, 6) (40,)


## 3. catch22 transform

### Univariate

The catch22 features are provided in the form of a transformer, `Catch22`.
From this the transformed data can be used for a variety of time series analysis tasks.

In [3]:
c22_uv = Catch22()
c22_uv.fit(IPD_X_train, IPD_y_train)

Catch22()

In [4]:
transformed_data_uv = c22_uv.transform(IPD_X_train)
print(transformed_data_uv.head())

SystemError: <built-in function DN_HistogramMode_5> returned a result with an error set

The transform `Catch22` method will process all 22 features.
For individual features, the transform_single_feature method can be used when provided with a numeric feature ID or the feature name.

In [ ]:
transformed_feature_uv = c22_uv._transform_single_feature(IPD_X_train,
                                                          "CO_f1ecac")
print(transformed_feature_uv)

### Multivariate

Transformation of multivariate data is supported by `Catch22`.
The default procedure will concatenate each column prior to transformation.

In [ ]:
c22_mv = Catch22()
c22_mv.fit(BM_X_train, BM_y_train)

In [ ]:
transformed_data_mv = c22_mv.transform(BM_X_train)
print(transformed_data_mv.head())

## 4. catch22 Forest Classifier

For classification tasks the default classifier to use with the catch22 features is random forest classifier.
An implementation making use of the `BaggingClassifier` from sklearn built on catch22 features is provided in the form on the `Catch22ForestClassifier` for ease of use.

In [ ]:
c22f = Catch22ForestClassifier(n_estimators=100)
c22f.fit(IPD_X_train, IPD_y_train)

In [ ]:
c22f_preds = c22f.predict(IPD_X_test)
print("C22F Accuracy: " + str(metrics.accuracy_score(IPD_y_test, c22f_preds)))